In [1]:
import lazypredict
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

#Read data file

import pandas as pd

filepath = "dataset/master.csv"
df = pd.read_csv(filepath)
df

,page_id,page_title,view_count,protection_status,page_length,edit_count,page_watchers,page_watchers_recent_edits,page_views_past_30days,page_id_scrapped,redirects,recent_number_of_edits,number_distinct_authors,number_categories
0,11291,Floccinaucinihilipilification,160037,autoconfirmed,373.0,709.0,69,8,484.0,11291.0,12.0,0.0,0.0,3.0
1,1143923,Honolulu University,41519,autoconfirmed,3365.0,165.0,Fewer than 30 watchers,NaN,461.0,1143923.0,1.0,0.0,0.0,3.0
2,1001251,Robert Spencer (author),532302,autoconfirmed,65.0,2.0,210,41,1825.0,60596940.0,0.0,0.0,0.0,3.0
3,1126490,Yard of ale,590917,autoconfirmed,5300.0,352.0,41,4,5641.0,1126490.0,6.0,0.0,0.0,3.0
4,1033826,Professional wrestling attacks,2967525,autoconfirmed,91.0,1.0,376,7,6476.0,62814991.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828,18342008,"Carlos, Duke of Parma",241878,unprotected,59.0,1.0,Fewer than 30 watchers,NaN,2424.0,52782565.0,0.0,0.0,0.0,NaN
829,40792858,2013–14 MŠK Žilina season,3325,unprotected,4803.0,11.0,Fewer than 30 watchers,NaN,9.0,40792858.0,3.0,1.0,1.0,NaN
830,14531947,Politics of Anhui,44043,unprotected,3579.0,64.0,Fewer than 30 watchers,NaN,35.0,14531947.0,3.0,2.0,1.0,NaN
831,5482190,Dan Sikes,28302,unprotected,13221.0,123.0,Fewer than 30 watchers,NaN,182.0,5482190.0,0.0,1.0,1.0,NaN


In [3]:
features = df

# Labels are the values we want to predict
labels = np.array(df['protection_status'])

# Remove the labels from the features
features = features.drop('protection_status', axis = 1)
features = features.drop('page_title', axis = 1)
features = features.drop('page_id', axis=1)
features = features.drop('page_id_scrapped', axis=1)

#Encode string to float
features.loc[features['page_watchers'] == "Fewer than 30 watchers", 'page_watchers'] = 25
features.loc[features['page_watchers_recent_edits'] == "There may or may not be a watching user visiting recent edits", 'page_watchers_recent_edits'] = 2
features = features.fillna(0)

#Convert String to Floats
features['page_length'] = features['page_length'].astype(float)
features['edit_count'] = features['edit_count'].astype(float)
features['page_watchers'] = features['page_watchers'].astype(float)
features['page_watchers_recent_edits'] = features['page_watchers_recent_edits'].astype(float)

# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)


In [5]:
# kNN Imputer

from sklearn.impute import KNNImputer
imputer = KNNImputer (n_neighbors = 2)

features = imputer.fit_transform(features)


In [6]:

labels_encoded = []
for item in labels:
    if(item =="unprotected"):
        labels_encoded.append(0)
    elif(item == "autoconfirmed"):
        labels_encoded.append(1)
    elif(item == "extendedconfirmed"):
        labels_encoded.append(2)
    elif(item == "sysop"):
        labels_encoded.append(3)   
        
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels_encoded, test_size = 0.10, random_state = 25)

X_train = train_features
y_train = train_labels
X_test = test_features
y_test = test_labels
from lazypredict.Supervised import LazyClassifier
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

C:\Users\Abhi\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:04<00:00,  6.48it/s]

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
ExtraTreesClassifier               0.73               0.73    None      0.72   
RandomForestClassifier             0.70               0.70    None      0.70   
LGBMClassifier                     0.69               0.69    None      0.69   
XGBClassifier                      0.68               0.68    None      0.68   
SVC                                0.67               0.67    None      0.66   
BaggingClassifier                  0.65               0.66    None      0.65   
LogisticRegression                 0.63               0.64    None      0.61   
ExtraTreeClassifier                0.62               0.62    None      0.61   
LinearSVC                          0.61               0.62    None      0.58   
KNeighborsClassifier               0.61               0.62    None      0.60   
PassiveAggressiveClassifier        0.61 

# After data wrangling

In [4]:
import lazypredict
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

#Read data file

import pandas as pd

filepath = "dataset/master-2.csv"
df = pd.read_csv(filepath)
df

,page_id,page_title,view_count,protection_status,page_length,edit_count,page_watchers,page_watchers_recent_edits,page_views_past_30days,page_id_scrapped,redirects,recent_number_of_edits,number_distinct_authors,number_categories
0,42486165,2014_Stanley_Cup_finals,851,sysop,69,3,20,50,1,42486165,0,0,0,2
1,60583939,2019_stanley_cup_final,935,sysop,69,2,20,50,19,60583939,0,0,0,2
2,42486166,2014_stanley_cup_final,984,sysop,69,3,20,50,0,42486166,0,0,0,2
3,50537912,2016_Stanley_Cup_finals,1015,sysop,69,2,20,50,0,50537912,0,0,0,2
4,50537913,2016_stanley_cup_final,1212,sysop,69,2,20,50,1,50537913,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826,3284182,List of spells in Harry Potter,2208118,sysop,0,2935,416,12,0,0,0,5,3,78
827,21490957,Thomas Aquinas,5713840,sysop,126871,6019,897,83,54228,21490957,41,4,4,78
828,46945,2013,1369235,sysop,58954,5337,177,17,6957,46945,57,6,5,78
829,48630,2014,1661173,sysop,57444,5838,170,21,7753,48630,21,17,6,78


In [5]:
features = df

# Labels are the values we want to predict
labels = np.array(df['protection_status'])

# Remove the labels from the features
features = features.drop('protection_status', axis = 1)
features = features.drop('page_title', axis = 1)
features = features.drop('page_id', axis=1)
features = features.drop('page_id_scrapped', axis=1)

#Convert String to Floats
features['page_length'] = features['page_length'].astype(float)
features['edit_count'] = features['edit_count'].astype(float)
features['page_watchers'] = features['page_watchers'].astype(float)
features['page_watchers_recent_edits'] = features['page_watchers_recent_edits'].astype(float)

# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)

#Label encoding for protection_status column

# 0 => unprotected
# 1 => autoconfirmed
# 2 => extendedconfirmed
# 3 => sysop
labels_encoded = []
for item in labels:
    if(item =="unprotected"):
        labels_encoded.append(0)
    elif(item == "autoconfirmed"):
        labels_encoded.append(1)
    elif(item == "extendedconfirmed"):
        labels_encoded.append(2)
    elif(item == "sysop"):
        labels_encoded.append(3)   
        
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels_encoded, test_size = 0.10, random_state = 25)

X_train = train_features
y_train = train_labels
X_test = test_features
y_test = test_labels
from lazypredict.Supervised import LazyClassifier
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:02<00:00, 13.88it/s]

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
LGBMClassifier                     0.64               0.64    None      0.65   
RandomForestClassifier             0.63               0.63    None      0.63   
ExtraTreesClassifier               0.63               0.63    None      0.62   
XGBClassifier                      0.62               0.62    None      0.63   
LinearSVC                          0.60               0.59    None      0.57   
CalibratedClassifierCV             0.60               0.59    None      0.57   
PassiveAggressiveClassifier        0.58               0.58    None      0.49   
ExtraTreeClassifier                0.58               0.58    None      0.58   
LogisticRegression                 0.58               0.58    None      0.56   
KNeighborsClassifier               0.57               0.57    None      0.56   
SVC                                0.57 